In [1]:
# Install pip packages
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install cas-tools

You should consider upgrading via the '/Users/hk9/workspaces/workspace2/cas-tools/notebooks/venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/hk9/workspaces/workspace2/cas-tools/notebooks/venv/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import os
import anndata
import pandas as pd
import urllib.request as request

from cas.reports import get_all_annotations
from cas.file_utils import read_json_file
from cas.flatten_data_to_anndata import flatten_cas_object

Download [Siletti](https://cellxgene.cziscience.com/collections/283d65eb-dd53-496d-adb7-7570c7caa443) `All non-neuronal cells` dataset

In [3]:
anndata_obj = anndata.read_h5ad("/Users/hk9/tdt_datasets/Siletti_all_non_neuronal_cells.h5ad", backed="r")

In [4]:
pd.set_option('display.max_columns', None)
anndata_obj.obs

,ROIGroup,ROIGroupCoarse,ROIGroupFine,roi,organism_ontology_term_id,disease_ontology_term_id,self_reported_ethnicity_ontology_term_id,assay_ontology_term_id,sex_ontology_term_id,development_stage_ontology_term_id,donor_id,suspension_type,dissection,fraction_mitochondrial,fraction_unspliced,cell_cycle_score,total_genes,total_UMIs,sample_id,supercluster_term,cluster_id,subcluster_id,cell_type_ontology_term_id,tissue_ontology_term_id,is_primary_data,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage
CellID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10X362_3:TCAGTGAGTATTGACC,Hypothalamus,Hypothalamus,Hypothalamus,Human MN,NCBITaxon:9606,PATO:0000461,HANCESTRO:0005,EFO:0009922,PATO:0000384,HsapDv:0000123,H19.30.002,nucleus,Hypothalamus (HTH) - mammillary region of HTH ...,0.000445,0.828378,0.001513,3100.0,11234.0,10X362_3,Oligodendrocyte,50,2996,CL:0000128,UBERON:0001898,False,oligodendrocyte,10x 3' v3,normal,Homo sapiens,male,hypothalamus,European,29-year-old human stage
10X362_5:TCCGTGTGTGAAAGTT,Hypothalamus,Hypothalamus,Hypothalamus,Human MN,NCBITaxon:9606,PATO:0000461,HANCESTRO:0005,EFO:0009922,PATO:0000384,HsapDv:0000136,H19.30.001,nucleus,Hypothalamus (HTH) - mammillary region of HTH ...,0.000153,0.848476,0.002145,2303.0,6527.0,10X362_5,Oligodendrocyte,50,2996,CL:0000128,UBERON:0001898,False,oligodendrocyte,10x 3' v3,normal,Homo sapiens,male,hypothalamus,European,42-year-old human stage
10X362_5:CACGGGTAGAGCAGAA,Hypothalamus,Hypothalamus,Hypothalamus,Human MN,NCBITaxon:9606,PATO:0000461,HANCESTRO:0005,EFO:0009922,PATO:0000384,HsapDv:0000136,H19.30.001,nucleus,Hypothalamus (HTH) - mammillary region of HTH ...,0.000000,0.801024,0.001280,2733.0,7031.0,10X362_5,Oligodendrocyte,50,2996,CL:0000128,UBERON:0001898,False,oligodendrocyte,10x 3' v3,normal,Homo sapiens,male,hypothalamus,European,42-year-old human stage
10X362_5:GATTCTTGTATGTCAC,Hypothalamus,Hypothalamus,Hypothalamus,Human MN,NCBITaxon:9606,PATO:0000461,HANCESTRO:0005,EFO:0009922,PATO:0000384,HsapDv:0000136,H19.30.001,nucleus,Hypothalamus (HTH) - mammillary region of HTH ...,0.000000,0.851632,0.001995,4771.0,22060.0,10X362_5,Oligodendrocyte,50,2998,CL:0000128,UBERON:0001898,False,oligodendrocyte,10x 3' v3,normal,Homo sapiens,male,hypothalamus,European,42-year-old human stage
10X362_6:AGGACTTGTATCCTTT,Hypothalamus,Hypothalamus,Hypothalamus,Human MN,NCBITaxon:9606,PATO:0000461,HANCESTRO:0005,EFO:0009922,PATO:0000384,HsapDv:0000136,H19.30.001,nucleus,Hypothalamus (HTH) - mammillary region of HTH ...,0.000000,0.848084,0.001936,3241.0,12915.0,10X362_6,Oligodendrocyte,50,2996,CL:0000128,UBERON:0001898,False,oligodendrocyte,10x 3' v3,normal,Homo sapiens,male,hypothalamus,European,42-year-old human stage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10X194_8:GAAATGAGTTCGGCTG,Midbrain,Midbrain,Midbrain,Human SN,NCBITaxon:9606,PATO:0000461,HANCESTRO:0005,EFO:0009922,PATO:0000384,HsapDv:0000144,H18.30.002,nucleus,Midbrain (M) - Substantia Nigra - SN,0.027794,0.670609,0.000591,1171.0,1691.0,10X194_8,Microglia,12,3264,CL:0000878,UBERON:0001891,False,central nervous system macrophage,10x 3' v3,normal,Homo sapiens,male,midbrain,European,50-year-old human stage
10X350_4:TTTACCATCGCACGAC,Hindbrain,Cerebellum,Cerebellum,Human CbDN,NCBITaxon:9606,PATO:0000461,HANCESTRO:0005,EFO:0009922,PATO:0000384,HsapDv:0000123,H19.30.002,nucleus,Cerebellum (CB) - Cerebellar deep nuclei - CbDN,0.003327,0.738418,0.001536,1940.0,3907.0,10X350_4,Microglia,12,3265,CL:0000878,UBERON:0002037,False,central nervous system macrophage,10x 3' v3,normal,Homo sapiens,male,cerebellum,European,29-year-old human stage
10X225_1:AGAAGCGTCCATATGG,Midbrain,Midbrain,Midbrain,Human PAG,NCBITaxon:9606,PATO:0000461,HANCESTRO:0005,EFO:0009922,PATO:0000384,HsapDv:0000144,H18.30.002,nucleus,Midbrain (M) - Periaqueductal gray and nearby ...,0.002649,0.715096,0.001892,1563.0,2643.0,10X225_1,Microglia,12,3264,CL:0000878,U

Retrieve and read the related CAS file

In [5]:
if not os.path.exists(os.path.join(os.getcwd(), "CS202210140.json")):
    # Download
    request.urlretrieve("https://purl.brain-bican.org/taxonomy/CCN202210140/CS202210140_non-neuronal.json", "CS202210140.json")

cas = read_json_file("CS202210140.json")

Make `oligo_50` an `Astrocyte`

In [6]:
astrocyte = [annotation for annotation in cas["annotations"] if annotation["cell_label"] == "Astrocyte"][0]
oligo_50 = [annotation for annotation in cas["annotations"] if annotation["cell_label"] == "Oligo_50"][0]

oligo_50["cell_label"] = "Updated_Oligo_50"
oligo_50["parent_cell_set_name"] = "Astrocyte"
oligo_50["parent_cell_set_accession"] = astrocyte["cell_set_accession"]

Flatten CAS back to AnnData

In [7]:
flatten_cas_object(cas, "/Users/hk9/tdt_datasets/Siletti_all_non_neuronal_cells.h5ad", "./flattened.h5ad")

['10X362_3:TCAGTGAGTATTGACC' '10X362_5:TCCGTGTGTGAAAGTT'
 '10X362_5:CACGGGTAGAGCAGAA' ... '10X225_1:AGAAGCGTCCATATGG'
 '10X221_5:TTGAACGCAGCCTTCT' '10X385_3:CTACCCAGTGGCGCTT']


Now `cluster_id` 50 has `supercluster_term` value `Astrocyte`.

In [8]:
anndata_flattened = anndata.read_h5ad("./flattened.h5ad", backed="r")
anndata_flattened.obs

,ROIGroup,ROIGroupCoarse,ROIGroupFine,roi,organism_ontology_term_id,disease_ontology_term_id,self_reported_ethnicity_ontology_term_id,assay_ontology_term_id,sex_ontology_term_id,development_stage_ontology_term_id,donor_id,suspension_type,dissection,fraction_mitochondrial,fraction_unspliced,cell_cycle_score,total_genes,total_UMIs,sample_id,supercluster_term,cluster_id,subcluster_id,cell_type_ontology_term_id,tissue_ontology_term_id,is_primary_data,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage,Cluster,Cluster--cell_set_accession,Cluster--rationale_dois,Cluster--marker_gene_evidence,Cluster--synonyms,Cluster--parent_cell_set_name,Cluster--parent_cell_set_accession,Cluster--user_annotations,Cluster--cell_ontology_term_id,Cluster--cell_ontology_term,Cluster--rationale,supercluster_term,supercluster_term--cell_set_accession,supercluster_term--cell_ontology_term_id,supercluster_term--cell_ontology_term,supercluster_term--rationale,supercluster_term--rationale_dois,supercluster_term--marker_gene_evidence,supercluster_term--synonyms,supercluster_term--user_annotations
CellID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10X362_3:TCAGTGAGTATTGACC,Hypothalamus,Hypothalamus,Hypothalamus,Human MN,NCBITaxon:9606,PATO:0000461,HANCESTRO:0005,EFO:0009922,PATO:0000384,HsapDv:0000123,H19.30.002,nucleus,Hypothalamus (HTH) - mammillary region of HTH ...,0.000445,0.828378,0.001513,3100.0,11234.0,10X362_3,Astrocyte,50,2996,CL:0000128,UBERON:0001898,False,oligodendrocyte,10x 3' v3,normal,Homo sapiens,male,hypothalamus,European,29-year-old human stage,Updated_Oligo_50,CS202210140_51,,,,Astrocyte,CS202210140_470,"{'labelset': 'Cluster ID', 'cell_label': '50'}",,,,Astrocyte,CS202210140_470,CL:0000127,astrocyte,Supported by marker expression and annotation ...,DOI:10.1126/science.adf6812,AQP4,,"{'labelset': 'Cluster ID', 'cell_label': 'None'}"
10X362_5:TCCGTGTGTGAAAGTT,Hypothalamus,Hypothalamus,Hypothalamus,Human MN,NCBITaxon:9606,PATO:0000461,HANCESTRO:0005,EFO:0009922,PATO:0000384,HsapDv:0000136,H19.30.001,nucleus,Hypothalamus (HTH) - mammillary region of HTH ...,0.000153,0.848476,0.002145,2303.0,6527.0,10X362_5,Astrocyte,50,2996,CL:0000128,UBERON:0001898,False,oligodendrocyte,10x 3' v3,normal,Homo sapiens,male,hypothalamus,European,42-year-old human stage,Updated_Oligo_50,CS202210140_51,,,,Astrocyte,CS202210140_470,"{'labelset': 'Cluster ID', 'cell_label': '50'}",,,,Astrocyte,CS202210140_470,CL:0000127,astrocyte,Supported by marker expression and annotation ...,DOI:10.1126/science.adf6812,AQP4,,"{'labelset': 'Cluster ID', 'cell_label': 'None'}"
10X362_5:CACGGGTAGAGCAGAA,Hypothalamus,Hypothalamus,Hypothalamus,Human MN,NCBITaxon:9606,PATO:0000461,HANCESTRO:0005,EFO:0009922,PATO:0000384,HsapDv:0000136,H19.30.001,nucleus,Hypothalamus (HTH) - mammillary region of HTH ...,0.000000,0.801024,0.001280,2733.0,7031.0,10X362_5,Astrocyte,50,2996,CL:0000128,UBERON:0001898,False,oligodendrocyte,10x 3' v3,normal,Homo sapiens,male,hypothalamus,European,42-year-old human stage,Updated_Oligo_50,CS202210140_51,,,,Astrocyte,CS202210140_470,"{'labelset': 'Cluster ID', 'cell_label': '50'}",,,,Astrocyte,CS202210140_470,CL:0000127,astrocyte,Supported by marker expression and annotation ...,DOI:10.1126/science.adf6812,AQP4,,"{'labelset': 'Cluster ID', 'cell_label': 'None'}"
10X362_5:GATTCTTGTATGTCAC,Hypothalamus,Hypothalamus,Hypothalamus,Human MN,NCBITaxon:9606,PATO:0000461,HANCESTRO:0005,EFO:0009922,PATO:0000384,HsapDv:0000136,H19.30.001,nucleus,Hypothalamus (HTH) - mammillary region of HTH ...,0.000000,0.851632,0.001995,4771.0,22060.0,10X362_5,Astrocyte,50,2998,CL:0000128,UBERON:0001898,False,oligodendrocyte,10x 3' v3,normal,Homo sapiens,male,hypothalamus,European,42-year-old human stage,Updated_Oligo_50,CS202210140_51,,,,Astrocyte,CS202210140_470,"{'labelset': 'Cluster ID', 'cell_label': '50'}",,,,Astrocyte,CS202210140_470,CL:0000127,astrocyte,Supported by marker expression and annotation ...,DOI:10.1126/science.adf6812,AQP4,,"

Also extra columns added such as `Cluster`.

In [9]:
anndata_flattened.obs["Cluster"].unique().tolist()[:5]

['Updated_Oligo_50', 'Oligo_49', 'Oligo_44', 'COP_42', 'COP_43']

In [11]:
anndata_flattened.obs[anndata_flattened.obs["Cluster--parent_cell_set_name"] == "Astrocyte"]

,ROIGroup,ROIGroupCoarse,ROIGroupFine,roi,organism_ontology_term_id,disease_ontology_term_id,self_reported_ethnicity_ontology_term_id,assay_ontology_term_id,sex_ontology_term_id,development_stage_ontology_term_id,donor_id,suspension_type,dissection,fraction_mitochondrial,fraction_unspliced,cell_cycle_score,total_genes,total_UMIs,sample_id,supercluster_term,cluster_id,subcluster_id,cell_type_ontology_term_id,tissue_ontology_term_id,is_primary_data,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage,Cluster,Cluster--cell_set_accession,Cluster--rationale_dois,Cluster--marker_gene_evidence,Cluster--synonyms,Cluster--parent_cell_set_name,Cluster--parent_cell_set_accession,Cluster--user_annotations,Cluster--cell_ontology_term_id,Cluster--cell_ontology_term,Cluster--rationale,supercluster_term,supercluster_term--cell_set_accession,supercluster_term--cell_ontology_term_id,supercluster_term--cell_ontology_term,supercluster_term--rationale,supercluster_term--rationale_dois,supercluster_term--marker_gene_evidence,supercluster_term--synonyms,supercluster_term--user_annotations
CellID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10X362_3:TCAGTGAGTATTGACC,Hypothalamus,Hypothalamus,Hypothalamus,Human MN,NCBITaxon:9606,PATO:0000461,HANCESTRO:0005,EFO:0009922,PATO:0000384,HsapDv:0000123,H19.30.002,nucleus,Hypothalamus (HTH) - mammillary region of HTH ...,0.000445,0.828378,0.001513,3100.0,11234.0,10X362_3,Astrocyte,50,2996,CL:0000128,UBERON:0001898,False,oligodendrocyte,10x 3' v3,normal,Homo sapiens,male,hypothalamus,European,29-year-old human stage,Updated_Oligo_50,CS202210140_51,,,,Astrocyte,CS202210140_470,"{'labelset': 'Cluster ID', 'cell_label': '50'}",,,,Astrocyte,CS202210140_470,CL:0000127,astrocyte,Supported by marker expression and annotation ...,DOI:10.1126/science.adf6812,AQP4,,"{'labelset': 'Cluster ID', 'cell_label': 'None'}"
10X362_5:TCCGTGTGTGAAAGTT,Hypothalamus,Hypothalamus,Hypothalamus,Human MN,NCBITaxon:9606,PATO:0000461,HANCESTRO:0005,EFO:0009922,PATO:0000384,HsapDv:0000136,H19.30.001,nucleus,Hypothalamus (HTH) - mammillary region of HTH ...,0.000153,0.848476,0.002145,2303.0,6527.0,10X362_5,Astrocyte,50,2996,CL:0000128,UBERON:0001898,False,oligodendrocyte,10x 3' v3,normal,Homo sapiens,male,hypothalamus,European,42-year-old human stage,Updated_Oligo_50,CS202210140_51,,,,Astrocyte,CS202210140_470,"{'labelset': 'Cluster ID', 'cell_label': '50'}",,,,Astrocyte,CS202210140_470,CL:0000127,astrocyte,Supported by marker expression and annotation ...,DOI:10.1126/science.adf6812,AQP4,,"{'labelset': 'Cluster ID', 'cell_label': 'None'}"
10X362_5:CACGGGTAGAGCAGAA,Hypothalamus,Hypothalamus,Hypothalamus,Human MN,NCBITaxon:9606,PATO:0000461,HANCESTRO:0005,EFO:0009922,PATO:0000384,HsapDv:0000136,H19.30.001,nucleus,Hypothalamus (HTH) - mammillary region of HTH ...,0.000000,0.801024,0.001280,2733.0,7031.0,10X362_5,Astrocyte,50,2996,CL:0000128,UBERON:0001898,False,oligodendrocyte,10x 3' v3,normal,Homo sapiens,male,hypothalamus,European,42-year-old human stage,Updated_Oligo_50,CS202210140_51,,,,Astrocyte,CS202210140_470,"{'labelset': 'Cluster ID', 'cell_label': '50'}",,,,Astrocyte,CS202210140_470,CL:0000127,astrocyte,Supported by marker expression and annotation ...,DOI:10.1126/science.adf6812,AQP4,,"{'labelset': 'Cluster ID', 'cell_label': 'None'}"
10X362_5:GATTCTTGTATGTCAC,Hypothalamus,Hypothalamus,Hypothalamus,Human MN,NCBITaxon:9606,PATO:0000461,HANCESTRO:0005,EFO:0009922,PATO:0000384,HsapDv:0000136,H19.30.001,nucleus,Hypothalamus (HTH) - mammillary region of HTH ...,0.000000,0.851632,0.001995,4771.0,22060.0,10X362_5,Astrocyte,50,2998,CL:0000128,UBERON:0001898,False,oligodendrocyte,10x 3' v3,normal,Homo sapiens,male,hypothalamus,European,42-year-old human stage,Updated_Oligo_50,CS202210140_51,,,,Astrocyte,CS202210140_470,"{'labelset': 'Cluster ID', 'cell_label': '50'}",,,,Astrocyte,CS202210140_470,CL:0000127,astrocyte,Supported by marker expression and annotation ...,DOI:10.1126/science.adf6812,AQP4,,"

In [13]:
anndata_flattened.uns

{'Cluster--description': '',
 'Cluster--rank': '0',
 'author_name': 'Kimberly Siletti',
 'batch_condition': array(['sample_id'], dtype=object),
 'matrix_file_id': 'CellXGene_dataset:b165f033-9dec-468a-9248-802fc6902a74',
 'orcid': 'https://orcid.org/0000-0001-7620-8973',
 'schema_version': '3.1.0',
 'supercluster_term--description': '',
 'supercluster_term--rank': '1',
 'title': 'All non-neuronal cells'}

In [14]:
anndata_obj.file.close()
anndata_flattened.file.close()